# 12 Root Finding (Students)

An important tool in the computational tool box is to find roots of equations for which no closed form solutions exist:

We want to find the roots $x_0$ of

$$
f(x_0) = 0
$$

## Problem: Projectile range 
The equations of motion for the projectile with linear air resistance (see *11 ODE applications*) can be solved exactly.

As a reminder: the linear drag force is
\begin{align}
\mathbf{F}_1 &= -b_1 \mathbf{v}\\
b &:= \frac{b_1}{m}
\end{align}

Equations of motion with force due to gravity $\mathbf{g} = -g \hat{\mathbf{e}}_y$

\begin{align}
\frac{d\mathbf{r}}{dt} &= \mathbf{v}\\
\frac{d\mathbf{v}}{dt} &= - g \hat{\mathbf{e}}_y -b \mathbf{v} 
\end{align}

### Analytical solution of the equations of motion
(Following Wang Ch 3.3.2)

Solve $x$ component of the velocity 

$$
\frac{dv_x}{dt} = -b v_x
$$

by integration:

$$
v_x(t) = v_{0x} \exp(-bt)
$$

The drag force reduces the forward velocity to 0.

Integrate again to get the $x(t)$ component

$$
x(t) = x_0 + \frac{v_{0x}}{b}\left[1 - \exp(-bt)\right]
$$

Integrating the $y$ component of the velocity

$$
\frac{dv_y}{dt} = -g - b v_y
$$

gives

$$
v_y = \left(v_{0y} + \frac{g}{b}\right) \exp(-bt) - \frac{g}{b}
$$

and integrating again

$$
y(t) = y_0 + \frac{v_{0y} + \frac{g}{b}}{b} \left[1 - \exp(-bt)\right] - \frac{g}{b} t
$$

(Note: This shows immediately that the *terminal velocity* is

$$
\lim_{t\rightarrow\infty} v_y(t) = - \frac{g}{b},
$$

i.e., the force of gravity is balanced by the drag force.)

#### Analytical trajectory 

To obtain the **trajectory $y(x)$** eliminate time (and for convenience, using the origin as the initial starting point, $x_0 = 0$ and $y_0 = 0$. Solve $x(t)$ for $t$

$$
t = -\frac{1}{b} \ln \left(1 - \frac{bx}{v_{0x}}\right)
$$

and insert into $y(t)$:

$$
y(x) = \frac{x}{v_{0x}} \left( v_{0y} + \frac{g}{b} \right) + \frac{g}{b^2} \ln \left(1 - \frac{bx}{v_{0x}}\right)
$$

#### Plot 

Plot the analytical solution $y(x)$ for $\theta = 30^\circ$ and $v_0 = 100$ m/s.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

The function `y_lindrag()` should compute $y(x)$. 

In [ ]:
def y_lindrag(x, v0, b1=0.2, g=9.81, m=0.5):
    b = b1/m
    v0x, v0y = v0
    # IMPLEMENT FUNCTION


def initial_v(v, theta):
    x = np.deg2rad(theta)
    return v * np.array([np.cos(x), np.sin(x)])

In [ ]:
X = np.concatenate([np.linspace(0, 42, 100), np.linspace(42, 45, 1000)])
Y = y_lindrag(X, initial_v(100, 30), b1=1)

In [ ]:
# PLOT

Compare to the numerical solution (from **11 ODE Applications**):

In [ ]:
import ode

def simulate(v0, h=0.01, b1=0.2, g=9.81, m=0.5):
    def f(t, y):
        # y = [x, y, vx, vy]
        return np.array([y[2], y[3], -b1/m * y[2], -g - b1/m * y[3]])

    vx, vy = v0
    t = 0
    positions = []
    y = np.array([0, 0, vx, vy], dtype=np.float64)
    
    while y[1] >= 0:
        positions.append([t, y[0], y[1]])  # record t, x and y
        t += h
        y[:] = ode.rk4(y, f, t, h)
        
    return np.array(positions)

In [ ]:
r = simulate(initial_v(100, 30), h=0.01, b1=1)

In [ ]:
plt.plot(X, Y, lw=2, label="analytical")
plt.plot(r[:, 1], r[:, 2], '--', label="RK4")
plt.legend(loc="best")
plt.xlabel("$x$ (m)"); plt.ylabel("$y$ (m)")

### Predict the range $R$
How far does the ball or projectile fly, i.e., that value $x=R$ where $y(R) = 0$:

$$
\frac{R}{v_{0x}} \left( v_{0y} + \frac{g}{b} \right) + \frac{g}{b^2} \ln \left(1 - \frac{bR}{v_{0x}}\right) = 0
$$

This *transcendental equation* can not be solved in terms of elementary functions.

Use a **root finding** algorithm.

## Root-finding with the Bisection algorithm
**Bisection** is the simplest (but very robust) root finding algorithm that uses trial-and-error:

* bracket the root
* refine the brackets
* see first part in [12_rootfinding.pdf (PDF)](12_rootfinding.pdf)

More specifically: 
1. determine a bracket that contains the root: $a < x_0 < b$ (i.e., an interval $[a, b]$ with $f(a) > 0$ and $f(b) < 0$ or $f(a) < 0$ and $f(b) > 0$)
2. cut bracket in half: $x' = \frac{1}{2}(a + b)$
3. determine in which half the root lies: either in $[a, x']$ or in $[x', b]$: If $f(a) f(x') > 0$ then the root lies in the right half $[x', b]$, otherwise the left half $[a, x']$.
4. Change the boundaries $a$ or $b$.
5. repeat until $|f(x')| < \epsilon$.

### Implementation of Bisection

- Test that the initial bracket contains a root; if not, raise a `ValueError`.
- If either of the bracket points is a root then return the bracket point.
- Allow `Nmax` iterations or until the convergence criterion `eps` is reached.
- BONUS: Raise a `RuntimeWarning` if no root was found after `Nmax` iterations, but print the best guess and the error.

  NOTE: It's better to fail with an exception than to return incorrect values. Calling code can always use `try... except` to catch your exception.

In [ ]:
def bisection(f, a, b, Nmax=100, eps=1e-14):
    # IMPLEMENT FUNCTION

### Finding the range with the bisection algorithm

Define the trial function:

Note that our `y_lindrag()` function depends on `x` **and** `v` but `bisect()` only accepts functions `f` that depend on a *single variable*, $f(x)$. We therefore have to wrap `y_lindrag(x, v)` into a function `f(x)` that sets `v` already to a value *outside* the function: [Python's scoping rules](https://stackoverflow.com/questions/291978/short-description-of-the-scoping-rules#292502) say that inside the function `f(x)`, the variable `x` has the value assigned to the argument of `f(x)` but any other variables such as `v` or `b1`, which were *not defined inside `f`*, will get the value that they had *outside `f`* in the *enclosing code*. 

```python
def f(x):
    v0 = initial_v(100, 30)
    b1 = 1.
    return y_lindrag(x, v0, b1=b1)
```

However, it is a bit messy to rely on using variables from outside the scope. It is cleaner to only use variables defined _inside_ the function. We can achieve this effect by using _keyword arguments_ where we set the default value to the value that we want to use inside our function: In this case, `b1` and `v0` are set as keyword arguments, with the understanding that when `f(x)` is called later, the keyword arguments are never changed and keep their defaults:

In [ ]:
v0 = initial_v(100, 30)
b1 = 1.0

def f(x, v0=v0, b1=b1):
    return y_lindrag(x, v0, b1=b1)

The initial bracket $[a_\text{initial}, b_\text{initial}]$ is a little bit difficult for this function: choose the right bracket near the point where the argument of the logarithm becomes 0 (which is actually the maximum $x$ value $\lim_{t\rightarrow +\infty} x(t) = \frac{v_{0x}}{b}$):

$$
b_\text{initial} = \frac{v_{0x}}{b} - \epsilon'
$$

where $\epsilon'$ is a small number.

In [ ]:
# use v0 and b1 from above!

m = 0.5
b = b1/m
# COMPLETE: bisection(... , eps=1e-6)

### Find the range as a function of the initial angle 

In [ ]:
b1 = 1.
m = 0.5
b = b1/m
speed = 100  # |v0| initial launch speed
u = []       # calculated tuples (angle, range)

# IMPLEMENT


In [ ]:
# PLOT

Write a function `find_range()` to calculate the range for a given initial velocity $v_0$ and plot $R(\theta)$ for $10\,\text{m/s} ≤ v_0 ≤ 100\,\text{m/s}$.

In [ ]:
def find_range(speed, b1=1, m=0.5):
    b = b1/m
    u = []
    for theta in np.arange(1, 90):
        v0 = initial_v(speed, theta)
        # IMPLEMENT THE REST ...
    return np.array(u)

In [ ]:
for speed in (10, 25, 50, 75, 100):
    u = find_range(speed)
    plt.plot(u[:, 0], u[:, 1], label="{} m/s".format(speed))
plt.legend(loc="best")
plt.xlabel(r"launch angle $\theta$")
plt.ylabel(r"range $R$ (m)")
# make labels with degree signs
ax = plt.gca()
ax.xaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter(r"{x:g}$\!^\circ$"))

## Newton-Raphson algorithm
(see derivation in class and in the second part of [12_rootfinding.pdf (PDF)](12_rootfinding.pdf)
 or [Newton's Method](http://mathworld.wolfram.com/NewtonsMethod.html) on MathWorld)


### Activity: Implement Newton-Raphson
1. Implement the Newton-Raphson algorithm

   `while` $|f(x)| > \epsilon$
   
   $\Delta x = -\frac{f(x))}{f'(x)}$
   
   $x \leftarrow x + \Delta x$
   
   Use a *finite difference* algorithm (which one?) with step size $h$ to compute $f'(x)$. (In other cases you might want to use the analytical derivative if it is available.)  
2. Test with $g(x)$.

   $$
   g(x) = 2 \cos x - x
   $$
   
3. Bonus: test performance of `newton_raphson()` and `bisection()` (e.g. for our function $g(x)$).
   * Under which circumstances is bisection faster than Newton-Raphson?
   * Can you *combine* the two algorithms to create a root finder that is faster than either by itself?

In [ ]:
def g(x):
    return 2*np.cos(x) - x

In [ ]:
xvals = np.linspace(0, 7, 30)
plt.plot(xvals, np.zeros_like(xvals), 'k--')
plt.plot(xvals, g(xvals))

In [ ]:
def newton_raphson(f, x, h=1e-3, Nmax=100, eps=1e-14):
    # IMPLEMENT ME

In [ ]:
newton_raphson(g, 0)